<a href="https://colab.research.google.com/github/luctiz/Grupo43-TP2/blob/master/TP2_Modelos_Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Modelos
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
#Ensambles
from sklearn.ensemble  import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, AdaBoostRegressor
#Búsqueda de hiper-parámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [0]:
#Cargo los features
features = pd.read_csv("../content/drive/My Drive/data/features_train.csv")
#features_test = pd.read_csv("../content/drive/My Drive/data/features_test.csv")

In [0]:
Y = pd.read_csv("../content/drive/My Drive/data/train.csv", usecols=['precio'])['precio']

In [0]:
X = features.iloc[:,:16] .join(features.iloc[:,16:].astype(np.int8))
del features

# Pruebo GridSearch

## (1) Modelo KNN

**Distancia Manhattan**

In [0]:
parameters_for_testing = {'metric': [ 'manhattan'], \
                          'weights': ['distance'], 'n_neighbors': [5,7,10]}
knn_model = KNeighborsRegressor()

In [0]:
gsearch1 = GridSearchCV(estimator = knn_model, param_grid = parameters_for_testing, n_jobs=1,iid=False, verbose=10,scoring='neg_mean_absolute_error')
gsearch1.fit(X,Y)



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] metric=manhattan, n_neighbors=5, weights=distance ...............
[CV]  metric=manhattan, n_neighbors=5, weights=distance, score=-799997.794, total= 6.7min
[CV] metric=manhattan, n_neighbors=5, weights=distance ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.7min remaining:    0.0s


[CV]  metric=manhattan, n_neighbors=5, weights=distance, score=-802631.732, total= 6.7min
[CV] metric=manhattan, n_neighbors=5, weights=distance ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 13.4min remaining:    0.0s


...

[CV]  metric=manhattan, n_neighbors=10, weights=distance, score=-803353.909, total= 7.6min
[CV] metric=manhattan, n_neighbors=10, weights=distance ..............


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 56.9min remaining:    0.0s


[CV]  metric=manhattan, n_neighbors=10, weights=distance, score=-809476.839, total= 7.6min


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 64.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 64.4min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid=False, n_jobs=1,
             param_grid={'metric': ['manhattan'], 'n_neighbors': [5, 7, 10],
                         'weights': ['distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=10)

In [0]:
print('best params')
print (gsearch1.best_params_)
print('best score')
print (-gsearch1.best_score_)

best params
{'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}
best score
801009.9628524253


**Como gridSearch tarda demasiado utilizo randomized Search**

# Pruebo RandomizedSearch

## KNN Regressor

In [0]:
from sklearn.model_selection import RandomizedSearchCV
parameters_for_testing = {'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'], \
                          'weights': ['uniform', 'distance'], 'n_neighbors': list(range(2,20)), 'n_jobs': [-1]}

n_iter_search = 20
random_search = RandomizedSearchCV(knn_model, param_distributions=parameters_for_testing, n_iter=n_iter_search, cv=5, iid=False, scoring='neg_mean_absolute_error')
random_search.fit(X,Y)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                                 metric='minkowski',
                                                 metric_params=None,
                                                 n_jobs=None, n_neighbors=5,
                                                 p=2, weights='uniform'),
                   iid=False, n_iter=20, n_jobs=None,
                   param_distributions={'metric': ['euclidean', 'manhattan',
                                                   'chebyshev', 'minkowski'],
                                        'n_jobs': [-1],
                                        'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9,
                                                        10, 11, 12, 13, 14, 15,
                                                        16, 17, 18, 19],
                                        'weights': ['uniform', 'distance']

In [0]:
print('best params')
print (random_search.best_params_)
print('best score')
print (-random_search.best_score_)

best params
{'weights': 'distance', 'n_neighbors': 12, 'n_jobs': -1, 'metric': 'manhattan'}
best score
792503.4736541035


## Gradient Boosting Regressor

In [0]:
gbr_model = GradientBoostingRegressor()

In [8]:
parameters_for_testing = {'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01], 'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200],\
                          'max_depth': np.linspace(1, 32, 32, endpoint=True), 'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),\
                          'max_features': list(range(1,X.shape[1])), 'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True)}

n_iter_search = 20
random_search = RandomizedSearchCV(gbr_model, param_distributions=parameters_for_testing, n_iter=n_iter_search, cv=3, iid=False, scoring='neg_mean_absolute_error', n_jobs = 2)
random_search.fit(X,Y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                            

In [9]:
print('best params')
print (random_search.best_params_)
print('best score')
print (-random_search.best_score_)

best params
{'n_estimators': 64, 'min_samples_split': 0.1, 'min_samples_leaf': 0.1, 'max_features': 222, 'max_depth': 7.0, 'learning_rate': 0.25}
best score
857630.7109325379


## XGBoost Regressor

In [0]:
 xgb_model = xgb.XGBRegressor()

In [12]:
parameters_for_testing = {'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01], 'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200],\
                          'max_depth': list(range(5,10)), 'min_child_weight':range(1,6,2), 'gamma':[i/10.0 for i in range(0,5)],\
                          'max_features': list(range(1,X.shape[1])), 'subsample':[i/10.0 for i in range(6,10)], \
                          'colsample_bytree':[i/10.0 for i in range(6,10)], 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]}

n_iter_search = 20
random_search = RandomizedSearchCV(xgb_model, param_distributions=parameters_for_testing, n_iter=n_iter_search, cv=3, iid=False, scoring='neg_mean_absolute_error', n_jobs = 2)
random_search.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[22:40:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_st...
                                        'max_features': [1, 2, 3, 4, 5, 6, 7, 8,
                                                         9, 10, 11, 12, 13, 14,
                            

In [13]:
print('best params')
print (random_search.best_params_)
print('best score')
print (-random_search.best_score_)

best params
{'subsample': 0.9, 'reg_alpha': 0.01, 'n_estimators': 100, 'min_child_weight': 1, 'max_features': 488, 'max_depth': 9, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.7}
best score
596734.5304386718


## Random Forest

In [0]:
rf_model = RandomForestRegressor(oob_score= True)

In [0]:
parameters_for_testing = {'n_estimators': range(10,100,10), 'random_state': range(1,50,7), \
                          'max_depth': range(5,50,5), 'min_samples_split': range(5,25,5), 'min_samples_leaf': [1,2,5,10,15]}

n_iter_search = 20
random_search = RandomizedSearchCV(rf_model, param_distributions=parameters_for_testing, n_iter=n_iter_search, cv=3, iid=False, scoring='neg_mean_absolute_error', n_jobs = 2)
random_search.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [0]:
print('best params')
print (random_search.best_params_)
print('best score')
print (-random_search.best_score_)